# Reranking (no Index support)

First we run the rerank subcommand to generate the top 10 reranked results. Note we have added a normalization parameter here to make sure all the features are normalized (so less chance of blowups).

    cd <scripts_dir>
    java -jar RankLib-2.10.jar \
        -load ../data/diy_lambdamart_model.txt \
        -rank ../data/diy_features_test.txt \
        -norm zscore \
        -score ../data/diy_lambdamart_scores.txt

Terminal output is as follows:

    Discard orig. features
    Model file:	../data/diy_lambdamart_model.txt
    Feature normalization: zscore
    Model:		LambdaMART
    Reading feature file [../data/diy_features_test.txt]... [Done.]            
    (5 ranked lists, 410 entries read)


In [1]:
import json
import operator
import os
import random
import requests
import urllib

In [2]:
DATA_DIR = "../../data"
SCORE_FILE = os.path.join(DATA_DIR, "diy_lambdamart_scores.txt")

SOLR_URL = "http://localhost:8983/solr/tmdbindex/"

QID_TO_QUERY = {
    16: "spy",
    17: "extramarital",
    18: "world war ii",
    19: "musical",
    20: "alien"
}
TOP_N = 10

In [3]:
def rating2label(rating):
    """ convert 0-10 continuous rating to 1-5 categorical labels """
    return int(rating // 2) + 1


def get_rating_string(rating):
    rating_string = []
    for i in range(rating):
        rating_string.append(u"\u2605")
    for i in range(5 - rating):
        rating_string.append(u"\u2606")
    return "".join(rating_string)


print(get_rating_string(3))
print(get_rating_string(rating2label(6.4)))

★★★☆☆
★★★★☆


## Top 20 without LTR

In [4]:
qid = random.randint(16, 20)
query = QID_TO_QUERY[qid]

In [5]:
def render_results(docs, query, top_n):
    print("top {:d} results for {:s}".format(top_n * 2, query))
    print("---")
    for doc in docs[0:top_n * 2]:
        doc_id = int(doc["id"])
        stars = get_rating_string(rating2label(float(doc["rating_f"])))
        score = float(doc["score"])
        title = doc["title_t"]
        print("{:s} {:06d} {:.3f} {:s}".format(stars, doc_id, score, title))


payload = {
    "q": query,
    "defType": "edismax",
    "qf": "title_t description_t",
    "pf": "title_t description_t",
    "mm": 2,
    "fl": "id,title_t,rating_f,score",            
    "rows": TOP_N * 5
}
params = urllib.parse.urlencode(payload, quote_via=urllib.parse.quote_plus)
search_url = SOLR_URL + "select?" + params
resp = requests.get(search_url)
resp_json = json.loads(resp.text)
docs = resp_json["response"]["docs"]
render_results(docs, query, TOP_N)

top 20 results for alien
---
★★★★☆ 000348 9.160 Alien
★★★★☆ 008077 9.160 Alien³
★★★☆☆ 008078 7.719 Alien: Resurrection
★★★☆☆ 029938 7.719 Alien Escape
★★★☆☆ 010128 7.719 Alien Nation
★★★☆☆ 009773 7.719 Alien Autopsy
★★★☆☆ 013954 7.719 Alien Raiders
★★★☆☆ 016634 7.719 Alien Trespass
★★★☆☆ 022076 7.719 Alien Hunter
★★★☆☆ 052318 7.719 Alien Predators
★★★☆☆ 258193 7.719 Alien Abduction
★★★☆☆ 052477 7.719 Alien Cargo
★★★☆☆ 312526 7.719 Alien Outpost
★☆☆☆☆ 113040 7.719 Alien Origin
★★☆☆☆ 104185 7.719 Alien Tornado
★★☆☆☆ 014907 7.719 Alien Apocalypse
★☆☆☆☆ 113119 7.719 Fugitive Alien
★★★☆☆ 126889 7.719 Alien: Covenant
★★★★☆ 067317 7.719 Dr. Alien
★★★☆☆ 044932 7.596 Without Warning


## Top 20 with LTR

In [6]:
fscores = open(SCORE_FILE, "r")
rows = []
for line in fscores:
    line = line.strip()
    qid, doc_id, score = line.split("\t")
    rows.append((int(doc_id), float(score)))
reranked_rows = sorted(rows, key=operator.itemgetter(1), reverse=True)[0:TOP_N]

In [7]:
reranked_docs = []
# LTR layer
for doc_id, score in reranked_rows:
    doc = docs[doc_id]
    doc["score"] = score
    reranked_docs.append(doc)
# rest of the results
doc_ids_to_remove = set([x[0] for x in reranked_rows])
for doc in docs:
    doc_id = int(doc["id"])
    if doc_id in doc_ids_to_remove:
        continue
    reranked_docs.append(doc)
    
render_results(docs, query, TOP_N)

top 20 results for alien
---
★★★★☆ 000348 0.028 Alien
★★★★☆ 008077 0.028 Alien³
★★★☆☆ 008078 0.028 Alien: Resurrection
★★★☆☆ 029938 0.028 Alien Escape
★★★☆☆ 010128 0.028 Alien Nation
★★★☆☆ 009773 0.028 Alien Autopsy
★★★☆☆ 013954 0.028 Alien Raiders
★★★☆☆ 016634 7.719 Alien Trespass
★★★☆☆ 022076 7.719 Alien Hunter
★★★☆☆ 052318 7.719 Alien Predators
★★★☆☆ 258193 7.719 Alien Abduction
★★★☆☆ 052477 7.719 Alien Cargo
★★★☆☆ 312526 7.719 Alien Outpost
★☆☆☆☆ 113040 7.719 Alien Origin
★★☆☆☆ 104185 7.719 Alien Tornado
★★☆☆☆ 014907 7.719 Alien Apocalypse
★☆☆☆☆ 113119 7.719 Fugitive Alien
★★★☆☆ 126889 7.719 Alien: Covenant
★★★★☆ 067317 7.719 Dr. Alien
★★★☆☆ 044932 7.596 Without Warning
